## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The example uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [1]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri("http://127.0.0.1:5001")

# mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5001'


In [3]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='mlflow-artifacts:/190105288112393163', creation_time=1761715763762, experiment_id='190105288112393163', last_update_time=1761715763762, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/154765283189832012', creation_time=1761711193887, experiment_id='154765283189832012', last_update_time=1761711193887, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1761614371564, experiment_id='0', last_update_time=1761614371564, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/10/29 05:32:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/29 05:32:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/190105288112393163/7d5f65754da045a0a341264b05a18f9b/artifacts'
🏃 View run carefree-doe-46 at: http://127.0.0.1:5001/#/experiments/190105288112393163/runs/7d5f65754da045a0a341264b05a18f9b
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/190105288112393163


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/190105288112393163', creation_time=1761715763762, experiment_id='190105288112393163', last_update_time=1761715763762, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/154765283189832012', creation_time=1761711193887, experiment_id='154765283189832012', last_update_time=1761711193887, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1761614371564, experiment_id='0', last_update_time=1761614371564, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>]

### Interacting with the model registry

In [9]:
from mlflow.tracking import MlflowClient

# mlflow.set_tracking_uri()
client = MlflowClient(f"http://127.0.0.1:5001")

In [10]:
client.search_registered_models()

[]

In [11]:
run_id = client.search_runs(experiment_ids=['1'])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

IndexError: list index out of range